### Gold Layer: Constructor Standings

Calculate constructor standings based on race results.

In [ ]:
dbutils.widgets.text("p_data_source", "")
v_data_source = dbutils.widgets.get("p_data_source")

In [ ]:
from formula1.formula1_constants import presentation_folder_path

##### Step 1 - Read race results from gold layer

In [ ]:
race_results_df = spark.read.parquet(f"{presentation_folder_path}/race_results")

##### Step 2 - Aggregate data to calculate points and wins

In [ ]:
from pyspark.sql.functions import sum, when, count, col

constructor_standings_df = race_results_df \
.groupBy("race_year", "team") \
.agg(sum("points").alias("total_points"),
     count(when(col("position") == 1, True)).alias("wins"))

In [ ]:
#display(constructor_standings_df.filter("race_year = 2020"))

##### Step 3 - Rank constructors by points and wins

In [ ]:
from pyspark.sql.window import Window
from pyspark.sql.functions import desc, rank, asc

constructor_rank_spec = Window.partitionBy("race_year").orderBy(desc("total_points"), desc("wins"))
final_df = constructor_standings_df.withColumn("rank", rank().over(constructor_rank_spec))

In [ ]:
#display(final_df.filter("race_year = 2020"))

##### Step 4 - Write to gold/presentation layer

In [ ]:
final_df.write.mode("overwrite").parquet(f"{presentation_folder_path}/constructor_standings")